# TEST KOLMOGOROV SMIRNOV

In [12]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Mon Sep 15 20:18:22 2025"

In [13]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),limit (Mb),max used,(Mb)
Ncells,811258,43.4,1462107,78.1,NA,1462107,78.1
Vcells,1552555,11.9,16505312,126.0,16384,20631435,157.5


In [14]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

In [15]:
PARAM <- list()
PARAM$semilla_primigenia <- 102191

PARAM$experimento <- 1999
PARAM$dataset <- "analistasr_competencia_2025.csv.gz"

#### Carpeta del Experimento

In [16]:
# carpeta de trabajo - Local version

# Use current project directory instead of Colab path
base_dir <- "/Users/martinschelp/dm2025b/src/workflows/experiments"
setwd(dirname(getwd()))  # Go to project root if needed

# Create experiments directory if it doesn't exist
dir.create(base_dir, recursive = TRUE, showWarnings = FALSE)
setwd(base_dir)

experimento_folder <- paste0("WF", PARAM$experimento)
dir.create(experimento_folder, showWarnings = FALSE)
setwd(file.path(base_dir, experimento_folder))

# Verify the directory was created
cat("Working directory set to:", getwd(), "\n")

Working directory set to: /Users/martinschelp/dm2025b/src/workflows/experiments/WF1999 


### 9.7.1   Preprocesamiento del dataset

#### 9.7.1.1  DT incorporar dataset

In [17]:
# lectura del dataset
dataset <- fread(paste0("/Users/martinschelp/dm2025b/datasets/datasets_gerencial_competencia_2025.csv.gz"))

In [18]:
View(dataset)


numero_de_cliente,foto_mes,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,...,ccallcenter_transacciones,chomebanking_transacciones,ctrx_quarter,Master_status,Master_fechaalta,Master_mpagominimo,Visa_status,Visa_fechaalta,Visa_mpagominimo,clase_ternaria
<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>
29186441,202005,1,61,282,2354.94,43761.59,1682.14,-139.02,712.60,...,0,20,47,0,6507,0.00,0,7671,1043.97,CONTINUA
29187961,202005,1,61,312,5066.31,47268.33,3875.62,-3536.03,4551.11,...,0,228,232,0,5939,2017.56,0,8090,7495.47,CONTINUA
29193101,202005,1,67,365,4809.42,37996.66,6982.73,-2962.24,1109.46,...,0,133,149,0,2026,2838.66,0,2026,1653.93,CONTINUA
29193281,202005,1,54,158,2979.52,88436.59,3350.01,-908.55,592.70,...,2,10,115,0,3125,844.56,0,3350,2029.29,CONTINUA
29198891,202005,1,54,312,3008.70,22567.32,273.68,-643.87,2975.52,...,0,124,32,0,7202,234.60,0,7242,1395.87,CONTINUA
29205441,202005,1,72,6,394.19,1607.93,159.49,-9.92,210.01,...,0,41,32,0,157,0.00,0,157,58.65,CONTINUA
29207901,202005,1,82,312,7080.40,50842.55,1895.97,-47.19,4466.99,...,0,24,123,NA,NA,NA,0,8480,0.00,CONTINUA
29243631,202005,1,54,110,2043.10,32934.54,492.60,915.30,406.53,...,0,87,81,0,992,0.00,0,3036,1395.87,CONTINUA
29251231,202005,1,82,312,1071.17,80557.53,635.61,-51.43,422.75,...,0,91,58,0,7298,304.98,0,8215,105.57,CONTINUA


In [22]:
# Kolmogorov-Smirnov Test for Data Drift Detection
# Comparing each foto_mes distribution against 202109 as reference

# Load required libraries
require("data.table")

# Parameters - adjust as needed
PARAM <- list()
PARAM$dataset <- ""  # Will be set based on your dataset
PARAM$reference_month <- 202109  # Reference month for comparison
PARAM$alpha <- 0.05  # Significance level

# Read dataset
cat("Loading dataset...\n")
dataset <- fread("/Users/martinschelp/dm2025b/datasets/datasets_gerencial_competencia_2025.csv.gz")

cat("Dataset loaded. Dimensions:", nrow(dataset), "x", ncol(dataset), "\n")

# Check available foto_mes values
foto_mes_values <- sort(unique(dataset$foto_mes))
cat("Available foto_mes values:", paste(foto_mes_values, collapse = ", "), "\n")

# Verify reference month exists
if (!PARAM$reference_month %in% foto_mes_values) {
  stop("Reference month ", PARAM$reference_month, " not found in dataset!")
}

# Get reference data (202109)
reference_data <- dataset[foto_mes == PARAM$reference_month]
cat("Reference month", PARAM$reference_month, "has", nrow(reference_data), "observations\n")

# Get monetary columns for KS testing
# por como armé los nombres de campos,
# estos son los campos que expresan variables monetarias
campos_monetarios <- colnames(dataset)
campos_monetarios <- campos_monetarios[campos_monetarios %like% "^(m|Visa_m|Master_m|vm_m)"]

# Use monetary columns as our test variables
numeric_cols <- campos_monetarios

cat("Testing", length(numeric_cols), "monetary variables\n")
cat("Monetary fields found:", paste(campos_monetarios, collapse = ", "), "\n")
cat("Variables to test:", paste(head(numeric_cols, 10), collapse = ", "), 
    if(length(numeric_cols) > 10) "..." else "", "\n")

# Function to perform KS test for a single variable between two foto_mes
perform_ks_test <- function(var_name, month1_data, month2_data, month1_name, month2_name) {
  # Remove NA values
  var1 <- month1_data[[var_name]]
  var2 <- month2_data[[var_name]]
  
  var1 <- var1[!is.na(var1)]
  var2 <- var2[!is.na(var2)]
  
  if (length(var1) == 0 || length(var2) == 0) {
    return(list(
      variable = var_name,
      comparison = paste(month1_name, "vs", month2_name),
      n1 = length(var1),
      n2 = length(var2),
      ks_statistic = NA,
      p_value = NA,
      significant = NA,
      interpretation = "Insufficient data"
    ))
  }
  
  # Perform KS test
  ks_result <- ks.test(var1, var2)
  
  return(list(
    variable = var_name,
    comparison = paste(month1_name, "vs", month2_name),
    n1 = length(var1),
    n2 = length(var2),
    ks_statistic = round(ks_result$statistic, 4),
    p_value = round(ks_result$p.value, 3),
    significant = ks_result$p.value < PARAM$alpha,
    interpretation = ifelse(ks_result$p.value < PARAM$alpha, 
                           "Significant difference (data drift detected)", 
                           "No significant difference")
  ))
}

# Perform KS tests for each foto_mes against reference
all_results <- list()
result_count <- 0

cat("\n=== PERFORMING KOLMOGOROV-SMIRNOV TESTS ===\n")

# Test each foto_mes (except reference) against reference month
test_months <- setdiff(foto_mes_values, PARAM$reference_month)

for (test_month in test_months) {
  cat("\nTesting", test_month, "vs", PARAM$reference_month, "\n")
  
  test_data <- dataset[foto_mes == test_month]
  cat("  Sample size:", nrow(test_data), "observations\n")
  
  # Test each numeric variable
  month_results <- list()
  significant_vars <- 0
  
  for (var_name in numeric_cols) {
    result_count <- result_count + 1
    
    ks_result <- perform_ks_test(var_name, test_data, reference_data, 
                                test_month, PARAM$reference_month)
    
    month_results[[var_name]] <- ks_result
    all_results[[result_count]] <- ks_result
    
    if (!is.na(ks_result$significant) && ks_result$significant) {
      significant_vars <- significant_vars + 1
    }
  }
  
  cat("  Variables with significant drift:", significant_vars, "out of", length(numeric_cols), "\n")
}

# Convert results to data.table for analysis
results_dt <- rbindlist(all_results)

# Summary by foto_mes
cat("\n=== SUMMARY BY FOTO_MES ===\n")
summary_by_month <- results_dt[!is.na(significant), .(
  total_variables = .N,
  significant_drift = sum(significant),
  pct_drift = round(100 * sum(significant) / .N, 1),
  min_p_value = min(p_value, na.rm = TRUE),
  max_ks_stat = max(ks_statistic, na.rm = TRUE)
), by = .(foto_mes = sub(" vs.*", "", comparison))]

print(summary_by_month)

# Variables with most drift across months
cat("\n=== VARIABLES WITH MOST FREQUENT DRIFT ===\n")
drift_by_variable <- results_dt[!is.na(significant), .(
  months_tested = .N,
  months_with_drift = sum(significant),
  pct_months_drift = round(100 * sum(significant) / .N, 1),
  avg_ks_statistic = round(mean(ks_statistic, na.rm = TRUE), 4),
  min_p_value = min(p_value, na.rm = TRUE)
), by = variable][order(-pct_months_drift, -avg_ks_statistic)]

print(head(drift_by_variable, 20))

# Most significant cases (lowest p-values)
cat("\n=== MOST SIGNIFICANT DRIFT CASES ===\n")
most_significant <- results_dt[significant == TRUE][order(p_value)][1:20]

# Format p_value to always show 3 decimals
most_significant_formatted <- most_significant[, .(
  variable, 
  comparison, 
  ks_statistic, 
  p_value = sprintf("%.3f", p_value)
)]
print(most_significant_formatted)

# Export results
output_file <- "ks_test_results.csv"
fwrite(results_dt, output_file)
cat("\nDetailed results exported to:", output_file, "\n")

# Summary statistics
total_tests <- nrow(results_dt[!is.na(significant)])
significant_tests <- nrow(results_dt[significant == TRUE])

cat("\n=== OVERALL SUMMARY ===\n")
cat("Total KS tests performed:", total_tests, "\n")
cat("Tests showing significant drift:", significant_tests, "\n")
cat("Percentage of tests with drift:", round(100 * significant_tests / total_tests, 1), "%\n")
cat("Reference month:", PARAM$reference_month, "\n")
cat("Significance level:", PARAM$alpha, "\n")

cat("\nInterpretation:")
cat("\n- KS test detects differences in distributions between months")
cat("\n- Significant results (p <", PARAM$alpha, ") indicate data drift")
cat("\n- Higher KS statistics indicate larger distributional differences\n")




Loading dataset...
Dataset loaded. Dimensions: 273666 x 32 
Available foto_mes values: 202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106, 202107, 202108, 202109 
Reference month 202109 has 16551 observations
Testing 14 monetary variables
Monetary fields found: mrentabilidad, mrentabilidad_annual, mcomisiones, mactivos_margen, mpasivos_margen, mcuenta_corriente, mcaja_ahorro, mcuentas_saldo, mtarjeta_visa_consumo, mtarjeta_master_consumo, mprestamos_personales, mpayroll, Master_mpagominimo, Visa_mpagominimo 
Variables to test: mrentabilidad, mrentabilidad_annual, mcomisiones, mactivos_margen, mpasivos_margen, mcuenta_corriente, mcaja_ahorro, mcuentas_saldo, mtarjeta_visa_consumo, mtarjeta_master_consumo ... 

=== PERFORMING KOLMOGOROV-SMIRNOV TESTS ===

Testing 202005 vs 202109 
  Sample size: 15082 observations


Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in th

  Variables with significant drift: 14 out of 14 

Testing 202006 vs 202109 
  Sample size: 15323 observations


Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in th

  Variables with significant drift: 14 out of 14 

Testing 202007 vs 202109 
  Sample size: 15522 observations


...

  Variables with significant drift: 13 out of 14 

Testing 202107 vs 202109 
  Sample size: 16551 observations


Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in th

  Variables with significant drift: 13 out of 14 

Testing 202108 vs 202109 
  Sample size: 16553 observations


Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in the presence of ties"
Warning message in ks.test.default(var1, var2):
"p-value will be approximate in th

  Variables with significant drift: 8 out of 14 

=== SUMMARY BY FOTO_MES ===
    foto_mes total_variables significant_drift pct_drift min_p_value
      <char>           <int>             <int>     <num>       <num>
 1:   202005              14                14     100.0           0
 2:   202006              14                14     100.0           0
 3:   202007              14                14     100.0           0
 4:   202008              14                14     100.0           0
 5:   202009              14                14     100.0           0
 6:   202010              14                14     100.0           0
 7:   202011              14                14     100.0           0
 8:   202012              14                14     100.0           0
 9:   202101              14                14     100.0           0
10:   202102              14                14     100.0           0
11:   202103              14                14     100.0           0
12:   202104             